# Advanced Tutorial on MetaSynth

In this tutorial, we will be creating a `generative metadata format` (`gmf`) metadata file from a dataset using MetaSynth. We are going to walk through some of the advanced abilities of MetaSynth, such as handling dates, setting distributions and ensuring uniqueness in columns. This example workflow starts from a `.csv` file as input, but it easily adapted to other formats. 

You can run this notebook by checking out the MetaSynth repo and installing metasynth with `pip install metasynth`

In [1]:
# %pip install metasynth

In [2]:
# import required packages
import datetime as dt
import polars as pl
from metasynth import MetaDataset
from utils import get_demonstration_fp

## Step 1: Transforming your data into a pandas DataFrame

The first step in creating the metadata is reading and converting your dataset to a pandas DataFrame. 

In [3]:
demonstration_fp = get_demonstration_fp()
df = pl.read_csv(demonstration_fp, parse_dates=True)
df.head()

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
i64,str,str,i64,i64,f64,str,str,date,time,datetime[μs]
1,"""Braund, Mr. Ow...","""male""",22,0,7.25,null,"""S""",1922-03-23,14:57:38,2022-08-13 08:42:37
2,"""Cumings, Mrs. ...","""female""",38,0,71.2833,"""C85""","""C""",1938-03-22,17:16:30,2022-07-24 06:45:36
3,"""Heikkinen, Mis...","""female""",26,0,7.925,null,"""S""",1909-11-06,15:57:59,2022-07-21 02:32:58
4,"""Futrelle, Mrs....","""female""",35,0,53.1,"""C123""","""S""",1915-10-14,14:17:36,2022-07-19 13:22:18
5,"""Allen, Mr. Wil...","""male""",35,0,8.05,null,"""S""",1929-04-18,13:23:39,2022-07-26 11:47:42


Now, let's check the data types of our DataFrame:

In [4]:
dict(zip(df.columns, df.dtypes))

{'PassengerId': polars.datatypes.Int64,
 'Name': polars.datatypes.Utf8,
 'Sex': polars.datatypes.Utf8,
 'Age': polars.datatypes.Int64,
 'Parch': polars.datatypes.Int64,
 'Fare': polars.datatypes.Float64,
 'Cabin': polars.datatypes.Utf8,
 'Embarked': polars.datatypes.Utf8,
 'Birthday': polars.datatypes.Date,
 'Board time': polars.datatypes.Time,
 'Married since': datetime[μs]}

We see that most variables are now nicely specified as strings, categories and ints where necessary. For the dates and times we just created, we see the dtype `object`. This is the "catch-all" dtype for pandas. But don't worry, these columns have the correct type and MetaSynth will deal with it correctly:

In [5]:
df["Birthday"][0]

datetime.date(1922, 3, 23)

## Step 2: Creating a MetaDataset object from a DataFrame

Now a lot of work has already gone into creating a properly formatted dataframe. This work pays off at this stage: let's convert the DataFrame to a meta_dataset structure with the default options. Note: this takes a little bit of time!

In [6]:
meta_dataset = MetaDataset.from_dataframe(df)

/home/qubix/Documents/shared_work/synthetic/meta-synth/metasynth/disttree.py:127: UserWarning: 
Variable PassengerId seems unique, but not set to be unique.
Set the variable to be either unique or not unique to remove this warning.

  warnings.warn(f"\nVariable {series.name} seems unique, but not set to be unique.\n"
/home/qubix/.pyenv/versions/3.10.4/lib/python3.10/site-packages/polars/internals/series/series.py:654: DeprecationWarning: passing a boolean mask to Series.__getitem__ is being deprecated; instead use Series.filter
  warnings.warn(


Then, we can show the metadata as a dictionary:

In [7]:
print(meta_dataset)

# Rows: 891
# Columns: 11

{'name': 'PassengerId', 'description': None, 'type': 'discrete', 'dtype': "<class 'polars.datatypes.Int64'>", 'prop_missing': 0.0, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 1, 'high': 892}}"}

{'name': 'Name', 'description': None, 'type': 'string', 'dtype': "<class 'polars.datatypes.Utf8'>", 'prop_missing': 0.0, 'distribution': '.[]{12,82}'}

{'name': 'Sex', 'description': None, 'type': 'string', 'dtype': "<class 'polars.datatypes.Utf8'>", 'prop_missing': 0.0, 'distribution': '[fm][ea][ml][ea][l][e]'}

{'name': 'Age', 'description': None, 'type': 'discrete', 'dtype': "<class 'polars.datatypes.Int64'>", 'prop_missing': 0.19865319865319866, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 0, 'high': 81}}"}

{'name': 'Parch', 'description': None, 'type': 'discrete', 'dtype': "<class 'polars.datatypes.Int64'>", 'prop_missing': 0.0, 'distribution': "{'name': 'PoissonDistribution', 'parameters': {'m

## Step 3: Saving the metadata in a file

After creating the metadata, we can save it to a file. The default format is `json`, meaning the file is quite legible by humans and computers alike. Therefore, it can be checked by the data controller and, when the disclosure risk is deemed to be low, this file can be shared with others.

In [8]:
file_path = "demonstration_metadata.json"
meta_dataset.to_json(file_path)

## Step 4: Generating synthetic data from the metadata

Upon receiving this file, you can use the MetaSynth package to generate a synthetic version of the dataset:

In [9]:
new_meta_dataset = MetaDataset.from_json(file_path)
new_meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
i64,str,str,i64,i64,f64,str,str,date,time,datetime[μs]
871,"""\[7n;9Sz[\p;iD...","""mamae""",61,0,52.167692,null,"""Q""",1936-01-04,16:47:49,2022-08-14 07:00:28
588,"""Z|`Vc!Ru0G""F u...","""memale""",1,0,61.734672,null,"""Q""",1930-11-25,10:59:40,2022-08-15 05:11:01
804,"""P`auBr@W9.$)9l...","""malee""",null,1,25.952398,"""Cl!b6 < u{0""","""S""",1914-04-17,14:19:31,2022-08-01 21:23:53
826,"""R>w&Fu!>""j>V}p...","""felale""",48,0,6.620778,"""B\NUy ayT""","""C""",1905-08-16,13:39:52,2022-07-24 19:02:27
45,"""Lf^ n<4r g~KB4...","""felele""",77,0,27.994717,null,"""C""",1918-12-10,11:36:54,2022-07-29 12:08:52


As you can see, the fake data looks a lot like the real data! However, it could still use some improvement. In the next sections, we will explore manual changes we can make to improve the quality of the synthetic data.

## Step 5: Improving the quality of the synthetic data

### Set unique columns

One column (PassengerId) has been detected as possibly unique by MetaSynth, as indicated by the following warning:

> "Variable PassengerId seems unique, but not set to be unique."

This column holds a variable with unique passenger identifiers, so in fact we do want synthetic data generated for this column to be unique as well. We can add this to the metadata by creating a list of options which we call a `specification`, or `spec`:

In [10]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}
}

# then, we add that dictionary as the `spec` argument
meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)

# then, let's check what the metadata about PassengerId contains!
meta_dataset["PassengerId"].to_dict()

/home/qubix/.pyenv/versions/3.10.4/lib/python3.10/site-packages/polars/internals/series/series.py:654: DeprecationWarning: passing a boolean mask to Series.__getitem__ is being deprecated; instead use Series.filter
  warnings.warn(


{'name': 'PassengerId',
 'type': 'discrete',
 'dtype': "<class 'polars.datatypes.Int64'>",
 'prop_missing': 0.0,
 'distribution': {'name': 'UniqueKeyDistribution',
  'parameters': {'low': 1, 'consecutive': 1}}}

So let's check what is generated from this new metadata:

In [11]:
meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
i64,str,str,i64,i64,f64,str,str,date,time,datetime[μs]
1,"""h!Wo$PG *u tlu...","""fala""",null,0,13.693465,"""Bdl^zFUA0Y^:F""","""C""",1934-04-14,15:57:25,2022-08-13 19:10:38
2,"""` elp }b^}GjIL...","""melee""",10,0,70.024721,null,"""Q""",1922-11-17,15:54:05,2022-08-09 03:29:40
3,"""IR cx7fCcCX)t|...","""malee""",11,0,30.651448,null,"""Q""",1910-01-23,17:48:45,2022-08-15 09:49:08
4,"""IHp*Zki7<kk$Th...","""femee""",null,0,9.51069,"""EG0""","""C""",1910-08-26,17:03:51,2022-07-18 23:50:01
5,"""R_|y E&JsRGCxf...","""felal""",10,0,9.594215,null,"""S""",1912-05-23,14:30:28,2022-08-05 23:43:54


### Fake names (and others)

As one can see, the `Name` of the passengers is not quite so well synthesized. The reason is that the string type interpreter in MetaSynth is designed for `structured` strings (like room numbers such as `B1.09`, `B1.01` or `A1.08`) and not unstructured strings. However, MetaSynth supports the [faker](https://faker.readthedocs.io/en/master/index.html) package, which includes a lot of data types that it can fake. The columns using faker are not based on the real data at all so they do not disclose any info about the real data.

We fake names as follows:

In [12]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

/home/qubix/.pyenv/versions/3.10.4/lib/python3.10/site-packages/polars/internals/series/series.py:654: DeprecationWarning: passing a boolean mask to Series.__getitem__ is being deprecated; instead use Series.filter
  warnings.warn(


PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
i64,str,str,i64,i64,f64,str,str,date,time,datetime[μs]
1,"""Jose Roach""","""famale""",74,0,19.942608,"""BYZs""","""S""",1922-10-24,11:29:41,2022-08-02 20:54:16
2,"""Mark Chang""","""femel""",21,0,14.684117,null,"""C""",1921-06-28,17:23:57,2022-07-18 21:11:36
3,"""Stephanie Smit...","""mala""",8,0,81.770468,null,"""S""",1915-10-28,17:27:22,2022-07-31 10:29:36
4,"""Joseph Lee""","""fale""",45,1,2.291116,null,"""C""",1939-09-11,11:35:39,2022-07-22 11:49:28
5,"""Joanne Knight""","""fale""",35,0,71.241426,null,"""S""",1927-03-31,17:56:42,2022-08-04 12:41:08


### Set distributions manually

Without user input, the distribution chosen for each variable is inferred by choosing the best fitting from available distributions for the variable type. However, we can also manually specify which distribution to fit, or we can even just fully specify how the variable should be generated.

In [13]:
from metasynth.distribution import DiscreteUniformDistribution

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)} # fully specify a distribution for age (uniform between 20 and 40)
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
i64,str,str,i64,i64,f64,f64,str,date,time,datetime[μs]
1,"""Thomas Summers...","""fala""",26,0,0.457304,null,"""Q""",1936-09-06,14:31:15,2022-07-26 06:50:40
2,"""Scott Gilmore""","""melee""",36,0,0.563613,null,"""C""",1914-12-08,14:37:08,2022-07-26 05:45:23
3,"""Sara Mccoy""","""memal""",36,3,1.11444,null,"""S""",1914-11-02,10:46:42,2022-07-24 08:35:54
4,"""Gail Quinn""","""mele""",31,0,0.689737,null,"""S""",1904-12-03,12:33:10,2022-08-01 20:50:37
5,"""David Steele""","""memee""",null,1,0.075603,null,"""S""",1932-04-22,18:06:05,2022-07-24 12:39:01


### Specifying the distribution of structured strings

For more or less structured strings, we can manually set the structure of the strings based on regular expressions. For example, we see that most Cabins are structured like [A-F] and then 2 or 3 digit numbers. We can include this as follows:

In [14]:
from metasynth.distribution import RegexDistribution

# To create a regex distribution, you need a list of tuples, where each tuple is an element.
# The first part of the tuple is a string representation of the regex, while the second is the proportion of the
# time the regex element is used.
cabin_distribution = RegexDistribution(r"[ABCDEF]\d{2,3}")  # Add the r so that it becomes a literal string.
# just for completeness: data generated from this distribution will always match the regex [ABCDEF]?(\d{2,3})?

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)}, # fully specify a distribution for age (uniform between 20 and 40)
    "Cabin": {"distribution": cabin_distribution}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(10)

PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
i64,str,str,i64,i64,f64,str,str,date,time,datetime[μs]
1,"""Joseph Lynch""","""fala""",29,0,6.745716,"""F247""","""S""",1914-11-02,17:10:57,2022-07-16 13:08:13
2,"""Jeanette Johns...","""memee""",34,0,1.848056,null,"""C""",1926-10-28,17:15:28,2022-08-07 01:50:05
3,"""Denise Singlet...","""malele""",32,1,0.149927,null,"""C""",1908-04-22,13:12:17,2022-07-21 18:15:03
4,"""Shannon Smith""","""mema""",null,0,0.254651,"""F775""","""S""",1921-03-21,13:55:39,2022-07-29 13:04:42
5,"""Karen Mitchell...","""felee""",21,0,1.385942,null,"""Q""",1907-02-19,13:06:58,2022-08-06 19:20:53
6,"""Laura Carter""","""melal""",26,1,0.220993,"""E604""","""C""",1915-09-17,17:11:47,2022-07-29 11:55:43
7,"""Lauren Tucker""","""mamal""",null,0,1.100163,"""A51""","""S""",1905-10-17,13:26:42,2022-07-26 10:44:15
8,"""Debra Spencer""","""malae""",38,0,1.283306,null,"""S""",1919-07-22,18:02:21,2022-07-16 14:33:29
9,"""Justin Palmer""","""mamal""",null,0,1.114592,"""B941""","""Q""",1915-12-18,13:07:51,2022-08-08 04:06:42
